In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import seaborn as sns
from keras.preprocessing import image
from tensorflow.keras.models import load_model

In [ ]:
from keras import datasets,layers,models


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
HFC_log_melspecs=np.load('/content/drive/MyDrive/dset/HFC_log_melspecs.txt.npy')
LFC_log_melspecs=np.load('/content/drive/MyDrive/dset/LFC_log_melspecs.txt.npy')

In [ ]:
HFC_log_melspecs.shape[0]
labels = [1]*HFC_log_melspecs.shape[0]
HFC_dataset_labeled = list(zip(HFC_log_melspecs , labels))
HFC_dataset_labeled = np.array(HFC_dataset_labeled)


<ipython-input-60-4d546138595c>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  HFC_dataset_labeled = np.array(HFC_dataset_labeled)


In [ ]:
LFC_log_melspecs.shape[0]
labels1 = [0]*LFC_log_melspecs.shape[0]
LFC_dataset_labeled = list(zip(LFC_log_melspecs , labels1))
LFC_dataset_labeled = np.array(LFC_dataset_labeled)

<ipython-input-61-a36492e5a474>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  LFC_dataset_labeled = np.array(LFC_dataset_labeled)


In [ ]:
LFC_dataset_labeled.shape

(952, 2)

In [ ]:
HFC_dataset_labeled.shape

(954, 2)

In [ ]:
dataset=np.concatenate((LFC_dataset_labeled,HFC_dataset_labeled),axis=0)
dataset.shape

(1906, 2)

In [ ]:
np.random.shuffle(dataset)

In [ ]:
y=[]
X=[]
for pt in dataset:
  y.append(pt[1])
  X.append(pt[0])

In [ ]:
X=np.array(X)
y=np.array(y)

In [ ]:
X=np.repeat(X[...,np.newaxis],3,axis=-1)

In [ ]:
X.shape

(1906, 200, 90, 3)

In [ ]:
y.shape

(1906,)

In [ ]:
min(y)

0

In [ ]:
max(y)

1

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,Y_test=train_test_split(X,y,test_size=0.2)

In [ ]:
X_train.shape

(1524, 200, 90, 3)

In [ ]:
X_test.shape

(382, 200, 90, 3)

In [ ]:
X_train[1][100][50]

array([0., 0., 0.])

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(200, 90, 3))
for layer in vgg_model.layers:
    layer.trainable = False


In [ ]:
model = tf.keras.models.Sequential([
   vgg_model,
   tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train, batch_size=32,epochs=10)

Epoch 1/10
48/48 [==============================] - 4s 58ms/step - loss: 0.6826 - accuracy: 0.8904
Epoch 2/10
48/48 [==============================] - 3s 58ms/step - loss: 0.0692 - accuracy: 0.9757
Epoch 3/10
48/48 [==============================] - 3s 57ms/step - loss: 0.0161 - accuracy: 0.9961
Epoch 4/10
48/48 [==============================] - 3s 58ms/step - loss: 0.0038 - accuracy: 0.9993
Epoch 5/10
48/48 [==============================] - 3s 59ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 6/10
48/48 [==============================] - 3s 60ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 7/10
48/48 [==============================] - 3s 60ms/step - loss: 8.2296e-04 - accuracy: 1.0000
Epoch 8/10
48/48 [==============================] - 3s 60ms/step - loss: 6.8387e-04 - accuracy: 1.0000
Epoch 9/10
48/48 [==============================] - 3s 59ms/step - loss: 5.4774e-04 - accuracy: 1.0000
Epoch 10/10
48/48 [==============================] - 3s 58ms/step - loss: 4.9509e-04 - accuracy: 

In [ ]:
model.evaluate(X_test,Y_test)

12/12 [==============================] - 1s 57ms/step - loss: 0.1021 - accuracy: 0.9660


[0.10208489745855331, 0.9659686088562012]

In [ ]:
y_pred=model.predict(X_test)

12/12 [==============================] - 1s 53ms/step


In [ ]:
y_pred

array([[1.0000000e+00, 2.4739281e-08, 5.5667924e-09],
       [2.0344661e-08, 1.0000000e+00, 2.0730060e-12],
       [9.9995899e-01, 4.0599516e-05, 3.6069997e-07],
       ...,
       [7.6271270e-08, 9.9999988e-01, 1.2774872e-09],
       [1.6899355e-08, 1.0000000e+00, 1.0838534e-11],
       [9.9997938e-01, 2.0604470e-05, 4.0259418e-09]], dtype=float32)

In [ ]:
import librosa
import librosa.display
from IPython.display import Audio,display as ipd
import matplotlib.pyplot as plt
import os
import numpy as np
from PIL import Image

In [ ]:
def scale_melspec(mel_spec):

  target_size = (200, 90)  # Target size for mel spectrograms
  # Scale mel spectrogram using PIL
  scaled_mel_spec = librosa.util.normalize(mel_spec)  # Normalize mel spectrogram
  scaled_mel_spec = (scaled_mel_spec * 255).astype(np.uint8)  # Convert to uint8
  scaled_mel_spec = Image.fromarray(scaled_mel_spec)  # Convert to PIL image
  scaled_mel_spec = scaled_mel_spec.resize(target_size, Image.ANTIALIAS)  # Resize
  scaled_mel_spec = np.array(scaled_mel_spec)
  print(scaled_mel_spec.shape)

  return scaled_mel_spec

In [ ]:
def predict(WAV):
  scale,sr=librosa.load(WAV)
  mel_spec=librosa.feature.melspectrogram(y=scale,sr=sr,n_fft=2048,hop_length=512,n_mels=90)
  mel_spec=scale_melspec(mel_spec)
  log_mel_spectrogram=librosa.power_to_db(mel_spec)
  log_mel_spectrogram=np.repeat(log_mel_spectrogram[...,np.newaxis],3,axis=-1)
  log_mel_spectrogram = np.expand_dims(log_mel_spectrogram, axis=0)
  print(log_mel_spectrogram.shape)
  prediction=model.predict(log_mel_spectrogram)
  classes={
      0:"healthy",
      1:"unhealthy",
      2:"distress"
  }
  pred=classes[np.argmax(prediction)]
  print(pred)


cls=predict('4.wav')
print(cls)


(224, 224)
(1, 224, 224, 3)


<ipython-input-85-cf2d5070714e>:8: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  scaled_mel_spec = scaled_mel_spec.resize(target_size, Image.ANTIALIAS)  # Resize


InvalidArgumentError: ignored